In [1]:
import os
import tqdm.notebook
import torch
import botorch
import gpytorch
import numpy as np
import pyDOE2

import matplotlib
from matplotlib import pyplot as plt

import fbbo

In [2]:
%load_ext autoreload
%autoreload 2

# Estimate standard deviation of all test function outputs

In [3]:
synthetic_problems = [
    ('Branin', {}),
    ('Eggholder', {}),
    ('GoldsteinPrice', {}),
    ('SixHumpCamel',{}),
    ('Hartmann3', {}),
    ('Ackley', {'d': 5}),
    ('Hartmann6', {}),
    ('Michalewicz', {'d': 10}),
    ('Rosenbrock', {'d': 10}),
    ('StyblinskiTang', {'d': 10}),
    ("Michalewicz", {"d": 5}),
    ("StyblinskiTang", {"d": 5}),
    ("Rosenbrock", {"d": 7}),
    ("StyblinskiTang", {"d": 7}),
    ("Ackley", {"d": 10}),
]

# Set the percentage of the range to use as the standard deviation of the noise

In [4]:
STDEV_PCT = 0.2

In [5]:
# storage for each problem and its noise stddev
problem_noise_dict = {}
# pnd = {
#     'd': {
#         2: {
#             'run_no': stdv
#         }
#     }
# }

In [7]:
# pregenerate 1 million LHS samples
lhs_samples = {}
for d in tqdm.notebook.trange(2, 11):
    lhs_samples[d] = pyDOE2.lhs(d, samples=100000)

  0%|          | 0/9 [00:00<?, ?it/s]

In [8]:
problems = synthetic_problems

for problem_name, problem_params in tqdm.notebook.tqdm(problems):
    if problem_name not in problem_noise_dict:
        problem_noise_dict[problem_name] = {}

    if "bbob" in problem_name:
        max_run_no = 15  # inclusve
    else:
        max_run_no = 1
    for i in range(1, max_run_no + 1):
        if max_run_no > 1:
            problem_params["run_no"] = i

        f = fbbo.util.test_func_getter(problem_name, problem_params)

        # extact info about the function
        lb, ub, d, fmin = f.lb, f.ub, f.dim, f.yopt.item()

        # rescale lhs samples to domain
        X = lhs_samples[d] * (ub - lb) + lb

        # evaluate f(X)
        try:
            fX = f(X)

        except ValueError:
            fX = [f(x) for x in X]

        dX = np.max(fX) - fmin

        print(problem_name, problem_params, dX)

        if d not in problem_noise_dict[problem_name]:
            problem_noise_dict[problem_name][d] = {}

        if "run_no" in problem_params:
            problem_noise_dict[problem_name][d][problem_params["run_no"]] = {
                "stdev": STDEV_PCT * dX
            }
        else:
            problem_noise_dict[problem_name][d]["stdev"] = STDEV_PCT * dX

  0%|          | 0/15 [00:00<?, ?it/s]

Branin {} 303.62421815471447
Eggholder {} 2002.5421167854306
GoldsteinPrice {} 1011973.8042217783
SixHumpCamel {} 162.62380351990475
Hartmann3 {} 3.8630560881723675
Ackley {'d': 5} 22.22939890683161
Hartmann6 {} 3.3223674027850363
Michalewicz {'d': 10} 9.660149998169544
Rosenbrock {'d': 10} 4355212.627842652
StyblinskiTang {'d': 10} 900.8315043115236
Michalewicz {'d': 5} 4.687658
StyblinskiTang {'d': 5} 587.4235626225984
Rosenbrock {'d': 7} 3762662.0778023107
StyblinskiTang {'d': 7} 796.4516432191913
Ackley {'d': 10} 22.116357576831536


In [9]:
problem_noise_dict

{'Branin': {2: {'stdev': 60.7248436309429}},
 'Eggholder': {2: {'stdev': 400.50842335708614}},
 'GoldsteinPrice': {2: {'stdev': 202394.76084435568}},
 'SixHumpCamel': {2: {'stdev': 32.52476070398095}},
 'Hartmann3': {3: {'stdev': 0.7726112176344735}},
 'Ackley': {5: {'stdev': 4.445879781366322}, 10: {'stdev': 4.423271515366308}},
 'Hartmann6': {6: {'stdev': 0.6644734805570073}},
 'Michalewicz': {10: {'stdev': 1.932029999633909}, 5: {'stdev': 0.9375316}},
 'Rosenbrock': {10: {'stdev': 871042.5255685304},
  7: {'stdev': 752532.4155604622}},
 'StyblinskiTang': {10: {'stdev': 180.16630086230475},
  5: {'stdev': 117.4847125245197},
  7: {'stdev': 159.29032864383828}}}

In [ ]:
np.savez(f'problem_noise_dict_{STDEV_PCT:g}.npz', pnd=problem_noise_dict)